# League of legend Winner team predict to Using Riot API

2019 데이터분석 개인 프로젝트 리메이크

리그 오브 레전드 경기 기록을 수집하고, 가공하여 승률에 영향을 주는 요인 분석과 실시간 스코어 변동에 따른 승률 예측 프로젝트

A project to collect league-of-legend game records to analyze the factors that affect victory and to predict winning rates based on real-time score fluctuations.

This kernel is for Koreans and I apologize to foreign viewers.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 2. Data Preprocessing / 데이터 전처리

## 라이브러리 로드 & 데이터 로드

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm_notebook # 오래 걸리는 작업 진행확인용

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
pd.options.display.max_columns = 60

In [ ]:
match = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/matchpre.pkl")

win_team_stat = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/win_team_stats.csv")

lose_team_stat = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lose_team_stats.csv")

win_team = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_winner_data.pkl")

lose_team = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_lose_data.pkl")

date = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lol_version_Date.csv")

In [ ]:
display(match,win_team,lose_team,win_team_stat,lose_team_stat)

In [ ]:
"""
Win, lose team 구분 확인
"""
for i in tqdm_notebook(range(len(win_team))):
    wf_valid = win_team['win'].iloc[i]
    
    if lose_team['win'].iloc[i] != wf_valid:
        pass
    else:
        print(str(i)+'데이터 정합성 문제')

In [ ]:
"""
팀 스탯 row 와 팀 기록 row 일치시키기
"""
gameId = win_team_stat["gameId"]
match = pd.merge(gameId,match,how="inner",on="gameId")
win_team = pd.merge(gameId,win_team,how="inner",on="gameId")
lose_team = pd.merge(gameId,lose_team,how="inner",on="gameId")


In [ ]:
display(match.shape,win_team.shape,lose_team.shape,win_team_stat.shape,lose_team_stat.shape)

90500개 이상이 남았다는 것은 게임 데이터 중복이 있다는 것이다.

In [ ]:
"""
duplicated() 함수는 리스트에 대한 중복검사를 지원하지 않으므로 리스트 칼럼 삭제
"""
match.drop("participants",axis=1,inplace=True)

win_team.drop("bans",axis=1,inplace=True)

lose_team.drop("bans",axis=1,inplace=True)

In [ ]:
display(match.shape,win_team.shape,lose_team.shape,win_team_stat.shape,lose_team_stat.shape)

In [ ]:
display(match.duplicated().sum(),win_team.duplicated().sum(),lose_team.duplicated().sum())

In [ ]:
display(win_team_stat.duplicated().sum(),lose_team_stat.duplicated().sum())

In [ ]:
match = match.drop_duplicates()
win_team = win_team.drop_duplicates()
lose_team = lose_team.drop_duplicates()

win_team_stat = win_team_stat.drop_duplicates()
lose_team_stat = lose_team_stat.drop_duplicates()

In [ ]:
display(match.shape,win_team.shape,lose_team.shape,win_team_stat.shape,lose_team_stat.shape)

In [ ]:
"""
win_team, lose_team으로 나누어 모든 테이블 병합
"""
win_team = pd.merge(match,win_team,how="left",on="gameId")
win_team = pd.merge(win_team,win_team_stat,how="left",on="gameId")

lose_team = pd.merge(match,lose_team,how="left",on="gameId")
lose_team = pd.merge(lose_team,lose_team_stat,how="left",on="gameId")
display(win_team,lose_team)

In [ ]:
del match
gc.collect()

In [ ]:
"""
팀 데이터셋을 전체 데이터로 병합하기 전에 칼럼명 일치시키기
"""
win_team.columns = win_team.columns.str.replace("win_","")

lose_team.columns = lose_team.columns.str.replace("lose_","")

In [ ]:
"""
전체 게임 정보 gamedata 테이블 생성
"""
gamedata = pd.concat([win_team,lose_team])
gamedata = gamedata.reset_index()
gamedata.drop("index",axis=1,inplace=True)
gamedata = gamedata.astype({"gameVersion":int})
gamedata = pd.merge(gamedata,date,how="inner",on="gameVersion")

In [ ]:
del win_team
del lose_team
gc.collect()

In [ ]:
gamedata

In [ ]:
"""
카테고리형 데이터(True,False) Label encoding / 분석에 용이하도록 가공
"""
bool_mapping = {True:1,False:0}
bool_col = gamedata.select_dtypes('bool').columns.tolist()

for col in bool_col:
    gamedata[col] = gamedata[col].map(bool_mapping)
    
win_mapping = {"Win":1,"Fail":0}
gamedata["win"] = gamedata["win"].map(win_mapping)

team_mapping = {100:"Blue",200:"Red"}
gamedata["teamId"] = gamedata["teamId"].map(team_mapping)

gamedata["date"] = pd.to_datetime(gamedata["date"])
gamedata["gameId"] = gamedata.astype({"gameId":object})

In [ ]:
gamedata

# 3. Data EDA(Exploratory Data Analysis) / 탐색적 데이터 분석

## 데이터 요약

In [ ]:
gamedata.info()

총 180928개의 게임 데이터가 있고( 승리팀 + 패배팀 ) 결측값은 없다.

이 칼럼들은 범주형 데이터이고  

firstBlood                      180928 non-null int64

firstTower                      180928 non-null int64

firstInhibitor                  180928 non-null int64

firstBaron                      180928 non-null int64

firstDragon                     180928 non-null int64

firstRiftHerald                 180928 non-null int64

이하 칼럼들은 실수형 데이터이다.

In [ ]:
gamedata.describe()

통계값을 살펴본 결과 크게 문제되는 아웃라이어나 값들은 없다. 

특별한 점은 전적관리를 위해 킬이 굉장히 중요하고 한 게임에 10킬 정도만 발생하는 천상계 게임들에서도 한 플레이어가 50킬 이상을 했다는것이 놀랍다. 

모든 값이 0인 vilemawKills , dominionVictoryScore 칼럼은 예측에 영향이 없으므로 제거

In [ ]:
gamedata.drop(["dominionVictoryScore","vilemawKills"],axis=1,inplace=True)

In [ ]:
display(gamedata[gamedata['win']==1].describe(),gamedata[gamedata['win']==0].describe())

통계를 통해 모든 데이터를 요약해보니, 굉장히 흥미롭다. 

먼저 전반적으로 승리팀 지표들의 평균이 패배팀 지표의 평균보다 높은 것을 볼 수 있으므로 

설문조사를 통해 가져온 칼럼들이 승패예측에 올바른 특성이라고 신뢰할 수 있다.

그리고 롤을 하면서 느꼈던 것들이 완전히 달라보인다.

1. firstblood는 승패에 생각보다 영향이 없음. 평균 0.6 대 0.4

2. firstinhibitor는 차이가 엄청나게 크다, 이는 억제기를 먼저 부수는 건 굉장히 중요하다고 볼 수 있다.

3. 타워킬은 아무래도 이긴팀이 더 많이 부쉈을 수밖에 없어서 엄청난 차이를 보이는 것 같다.

4. 바론의 승패팀 차이와, 용의 승패팀 차이를 보았을때, 용을 먹은 것이 훨씬 차이가 크다. 이는 바론보단 용을 중심으로 챙겨야한다는 것을 뜻한다.

5. 용, 바론, 전령의 우선순위는 용 > 바론 > 전령이다.

6. 승리팀들의 킬 평균은 5.6, 데스 평균은 3.3 정도이다. 모든 게임에 적용되는 것은 아니겠지만 한 게임당 최소 KD 1.6 을 지향하면 승률이 상승할 것 같다.(충분히 경기수가 많으므로 대표성을 띌 수 있다)

7. 승리팀들의 딜량 평균은 14100-14300 사이이고, 진팀은 11600 - 11800 사이이다. 25분 기준, 팀에서 딜량 14000정도를 넣는다면 1인분을 한 것이다.

8. 25분 기준 10100 골드 또한 1인분 기준이 될 수 있다.

9. 시야점수는 생각보다 차이가 굉장히 적다(3) , 롤에서 시야가 중요하다고 생각했는데, 시야점수는 승리에 영향을 크게 미치지 않는 것같다. 하지만 25분 기준 시야점수 24는 넘도록 해야겠다.

10. CC기의 영향도 그렇게 크진 않지만(14), 없는 것보단 낫다는 것이 데이터로 증명되었다.


In [ ]:
gamedata["date"].describe()

추가한 날짜 칼럼을 통해 현재 데이터셋은 2018-12-15 일자부터 2020-03-25 일까지의 게임이란 것을 알 수 있다.

## 데이터 시각화

In [ ]:
gameoverview = gamedata.drop('gameVersion',axis=1)

In [ ]:
plt.figure(figsize=(20,12))
gameoverview.boxplot(vert=0)

데이터들이 같은 단위를 가지는 데이터에서 큰 차이가 없이 모여있는 것을 보아 아웃라이어는 없다고 판단해도 될 것 같다.

In [ ]:
"""
각 변수별 상관계수 시각화
"""
corr = gamedata.corr()

sns.clustermap(corr,cmap = 'RdYlBu_r', vmin = -1, vmax = 1)

### 날짜

In [ ]:
gamedata["year"] = gamedata["date"].dt.year
gamedata["month"] = gamedata["date"].dt.month
gamedata["day"] = gamedata["date"].dt.day

In [ ]:
f,ax = plt.subplots(3,1,figsize=(20,12))
sns.countplot(gamedata["year"],ax=ax[0])
sns.countplot(gamedata["month"],ax=ax[1])
sns.countplot(gamedata["day"],ax=ax[2])

최근 3개월간을 중심으로 2020 시즌에 대한 게임이 많이 수집되었다. 또한 공식적으로 패치날짜로 명시되지 않은 날짜에 잠수함 패치가 여러번 있었다는 것을 알게되었다.

### 게임 시간 / 경기 기록

게임 진행 시간은 각 경기 기록과 상관관계가 높은 칼럼이므로 다른 칼럼과 연계하여 분석한다.

In [ ]:
#gameDuration 은 게임시작시부터 끝까지 초단위로 기록되어있다. 
gamedata["gameMinute"] = gamedata["gameDuration"] / 60

In [ ]:
plt.figure(figsize=(20,12))
sns.distplot(gamedata["gameMinute"],kde=True)

게임 시간을 보니 3-4 분대의 기록이 존재한다. 3-4분대는 게임의 다시하기 기능으로, 팀원이 탈주, 에러가 발생했을 경우 투표로 5분내 게임을 무효로 종료할 수 있다.
따라서 해당 데이터는 승패 예측에 영향을 줄 수 없으므로 제거해야함.

In [ ]:
gamedata = gamedata[gamedata["gameMinute"] > 5]

### 경기 시간별 구간화

게임 시간에 따른 경기 기록을 분석하기 위해 구간을 나눠서 분석하기로 결정,

다시하기 게임을 제외한 15분 이전, 15-20분, 20-25분, 25-30분, 30-35분, 35분 이후 게임 순으로 분석 

In [ ]:
gamedata[(gamedata["gameMinute"] < 15)]

LOL의 시스템은 다시하기를 제외하고 15분 이전의 서렌(항복)은 허용되지 않는다. 15분 이전에 넥서스(상대 기지)를 부수는 것은 매우 힘든 일이고 거의 일어나지 않는다.

하지만 약 7000개, 전체데이터의 4%가 이것이 가능한 일임을 나타낸다. 또한 이것이 각 팀의 실력이 가장 비슷하고 게임 포기가 거의 없는 챌린저, 그랜드마스터, 마스터 티어의 경기기록임을

고려하면 하위 티어(팀 간의 실력 편차가 크고 게임 포기가 잦음)에서는 15분 전 게임 종료가 더욱 빈번할 것임이 추측된다.

In [ ]:
"""
해당 게임의 blue,red팀의 정보를 확인하기 위해 gameId를 라벨인코딩(조건문에서 gameId 값이 인식안됨)
"""
from sklearn.preprocessing import LabelEncoder
gamedata["gameId"] = LabelEncoder().fit_transform(gamedata["gameId"])
gamedata["gameId"].value_counts()

In [ ]:
earlygame = gamedata[(gamedata["gameMinute"] < 15)]
earlygame

In [ ]:
display(earlygame[earlygame["gameId"]==88419],earlygame[earlygame["gameId"]==87909],earlygame[earlygame["gameId"]==88253])

In [ ]:
display(earlygame[earlygame['win']==1].describe(),earlygame[earlygame['win']==0].describe())

15분대의 모든 지표가 엄청난 차이를 보이고 있다. 오브젝트형 칼럼들은 압도적이고, 킬또한 2.8-2.9 , 데스 0.7-0.8로 K/D가 4가 된다.

확실하게 보기 위해 시각화하여 정리한다.

In [ ]:
columns = gamedata.columns.tolist()

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
early_win = earlygame[earlygame["win"]==1]
early_lose = earlygame[earlygame["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(earlygame.iloc[:,i],bins=50)
    ax[j].hist(early_win.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(early_lose.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

15분 이전의 게임은 롤에서 초반, 라인전(1vs) 중심의 전개이다.

15분 이전에 끝난 게임들은, 먼저 지표들의 차이가 굉장히 극명하게 나타난다.

또한 15분내 포탑을 먼저 부수거나(firstTower), 억제기를 먼저 부수거나(firstInhibitor), 포탑을 2개 이상 부순(towerKills) 경우는 100% 확률로 승리한 것을 볼 수 있다.
이는 향후 게임의 승률을 높일때 최우선적으로 고려해야 할 사항이란 것을 알 수 있다. 

In [ ]:
game15_20 = gamedata[(gamedata["gameMinute"] < 20)&(gamedata["gameMinute"]>=15)]
game15_20

In [ ]:
display(game15_20[game15_20['win']==1].describe(),game15_20[game15_20['win']==0].describe())

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
win_15_20 = game15_20[game15_20["win"]==1]
lose_15_20 = game15_20[game15_20["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(game15_20.iloc[:,i],bins=50)
    ax[j].hist(win_15_20.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(lose_15_20.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

롤에서 15- 20분은 초중반 대의 게임이다.

15-20분 사이의 게임은 지표들의 차이가 조금 줄어들었다. 이 시간대부터 확실하게 100% 승리를 보장하는 지표는 없다.

하지만 firstTower(최초 포탑 파괴)여부, dragonKills(용 처치 횟수) 2회 이상, death1-5(죽음 횟수) 0회가 승리할 확률이 높다고 판단할 수 있다.

In [ ]:
game20_25 = gamedata[(gamedata["gameMinute"] < 25)&(gamedata["gameMinute"]>=20)]
game20_25

In [ ]:
display(game20_25[game20_25['win']==1].describe(),game20_25[game20_25['win']==0].describe())

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
win_20_25 = game20_25[game20_25["win"]==1]
lose_20_25 = game20_25[game20_25["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(game20_25.iloc[:,i],bins=50)
    ax[j].hist(win_20_25.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(lose_20_25.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

롤에서 20-25분은 중간대의 게임이다.

20-25분대의 게임은 totalTimeCrowdControlDealt( 군중제어기 사용시간 ) 지표는 거의 완전히 겹쳐지기 시작했고, firstBaron, baronKills 바론(20분 후부터 등장) 처치여부의 변화가 나타나기 시작했다.

이 시간대에 중요하게 보아야할 지표는, firstinhibitor(최초 억제기 파괴), riftHeraldKills(전령처치 횟수) 2회 이상, dragonKills(용  처치횟수) 3회 이상이 승률이 높다고 판단된다.

In [ ]:
game20_25.pivot_table(index="win", columns="firstBaron", aggfunc="size")

또한 20-25분대에 바론을 먹으면 약 97%확률로 승리한다는 것을 알 수 있다.

In [ ]:
game25_30 = gamedata[(gamedata["gameMinute"] < 30)&(gamedata["gameMinute"]>=25)]
game25_30

In [ ]:
display(game25_30[game25_30['win']==1].describe(),game25_30[game25_30['win']==0].describe())

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
win_25_30 = game25_30[game25_30["win"]==1]
lose_25_30 = game25_30[game25_30["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(game25_30.iloc[:,i],bins=50)
    ax[j].hist(win_25_30.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(lose_25_30.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

25-30분대는 롤에서 중후반으로 접어든 게임이라고 한다. 

군중제어기 시간은 20-25분대에서는 이제 승리에 거의 영향이 없고 totalDamageDealt(챔피언에게 가한 피해)도 겹쳐지는 것을 봐선 딜량도 승률에 미치는 영향이 떨어져가고 있다. 

이것이 의미하는 바는, 위의 지표들이 구분되던 10-25분대는 라인전(1vs1) 중심의 싸움이 주고, 25-30분대는 한타(5vs5) 중심의 싸움이 주가 되어 개인이 팀의 승률에 미치는 영향이 많이 떨어졌다고 판단된다.

그리고, 놀라운 점은 다른 first(최초) 관련 지표는 승리에 미치는 영향이 줄어드는 반면, firstinhibitor(최초 억제기 파괴)는 25-30분 시간대에도 지표가 유지되고 있다.
이는 30분대까지 억제기를 많이 파괴할수록 다른 어떤 것보다 승리할 확률이 굉장히 높아진다는 것이다.

다른 승리에 영향을 줄 수 있는 지표는 dragonKills(드래곤 처치 횟수) 4회, death(죽음) 횟수가 적은것이다.

In [ ]:
game25_30.pivot_table(index="win", columns="firstBaron", aggfunc="size")

또한 바론을 먹는 것도 90% 확률로 승리가 가능한 것을 볼 수 있다.

In [ ]:
game30_35 = gamedata[(gamedata["gameMinute"] < 35)&(gamedata["gameMinute"]>=30)]
game30_35

In [ ]:
display(game30_35[game30_35['win']==1].describe(),game30_35[game30_35['win']==0].describe())

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
win_30_35 = game30_35[game30_35["win"]==1]
lose_30_35 = game30_35[game30_35["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(game30_35.iloc[:,i],bins=50)
    ax[j].hist(win_30_35.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(lose_30_35.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

30-35분 대는 후반부 게임으로 접어든 때이다. 대부분의 개인 기록들은 승패에 영향을 끼치지 못하고, goldEarnend(획득한 골드)만이 그나마 구분의 지표가 된다. 

억제기를 두개 이상 파괴하거나, 드래곤을 4번 이상 처치하거나 바론을 2번 이상 처치하는 등, 오브젝트들의 지표로 승리가 결정된다.

따라서 후반부에는 오브젝트 중심의 운영이 필요하며 개인보다 팀 위주의 싸움을 중심적으로 해야 한다.

In [ ]:
late_game = gamedata[(gamedata["gameMinute"] >= 35)]
late_game

In [ ]:
"""
게임 기록 win,lose 나눠서 시각화(빨강-lose, 파랑-win)
"""
f, axes = plt.subplots(20,2,figsize=(10,20))
late_win = late_game[late_game["win"]==1]
late_lose = late_game[late_game["win"]==0]

ax = axes.ravel()

for i,j in zip(range(5,45),range(0,40)):
    _,bins = np.histogram(late_game.iloc[:,i],bins=50)
    ax[j].hist(late_win.iloc[:,i],bins=bins,color="blue",alpha=0.5)
    ax[j].hist(late_lose.iloc[:,i],bins=bins,color="red",alpha=0.5)
    ax[j].set_title(columns[i])
    ax[j].set_yticks(())
f.tight_layout()

35분 이후 게임은 극후반대 게임으로 분류하는데, 거의 모든 지표가 승패를 예측할 수 없게 되었다.

이는 극후반대 게임에 진입하면 게임 기록 상의 스코어보다 외부적인, 플레이어의 컨디션에 따른 것으로 보인다. 

만약 후반부 게임을 진행할 시, 지표를 보면서 플레이하기보다, 포탑 파괴를 중점으로 운영하되 그때 상황에 맞춰서 하는 유동적인 플레이가 중요할 것으로 보인다.

## 분석 정리

In [ ]:
#승패에 영향이 가는 상관계수
gamedata.corr()["win"]

1. 리그 오브 레전드에서 초반(0-15), 초중반(15-20), 중반(20-25), 중후반(25-30), 후반(30-35), 극후반(35-inf)대 별로 중요한 지표가 완전히 달라지는 것을 볼 수 있었다.

2. 따라서 경기 시간마다 바라봐야할 지표가 완전히 다르다는 것을 알게 되었다.

3. 승패 예측에 있어서는 개인 단위의 기록들보다 팀 단위의 기록들이 좀 더 도움이 될 것이라 추측.

4. 승리팀, 패배팀 간의 지표 차이또한 중요한 요소가 될 수 있을 것으로 보인다. -(당연하지만, 지표 차이가 클수록 이길 확률이 굉장히 커짐 )

5. 시간대를 승패에 영향이 가도록 가공하면 좀 더 신뢰성을 띄는 모델을 만들 수 있을 거라 예상.

6. 


이 정보들을 통해, 특성 공학에 필요한 정보를 얻었고 앞으로 진행할 모델링에 큰 도움이 될 것이다. 